<a href="https://colab.research.google.com/github/boristrs/seq2seq-fingerprint_MI/blob/master/RegressionBenchmark_v_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deepchem

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.2/693.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 58.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.4.23 requires scipy>=1.9, but you have scipy 1.8.1 which is incompatible.
jaxlib 0.4.23+cuda12.cudnn89 requires scipy>=1.9, but you have scipy 1.8.1 which is incompatible.


In [12]:
!pip install torch_geometric pytorch_lightning
#!pip install tensorflow==1.4.1
#!pip install haiku

ERROR: Could not find a version that satisfies the requirement tensorflow==1.4.1 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.16.0rc0)
ERROR: No matching distribution found for tensorflow==1.4.1


In [3]:
import tensorflow as tf
import deepchem as dc

import numpy as np

np.random.seed(123)

from deepchem.feat import Featurizer

/usr/local/lib/python3.10/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


In [6]:
!git clone https://github.com/boristrs/seq2seq-fingerprint_MI.git

fatal: destination path 'seq2seq-fingerprint_MI' already exists and is not an empty directory.


In [7]:
!pip freeze

absl-py==1.4.0
aiohttp==3.9.3
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.6.0
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array-record==0.5.0
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.0
bigframes==0.21.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.42
branca==0.7.1
build==1.0.3
CacheControl==0.14.0
cachetools==5.3.2
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.85
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.1
colorcet==3.0.1
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.0
cryptography==42.0.3
cufflinks==0.17.3
cupy-cuda12x==12.2.0


In [31]:
import sys
sys.path.insert(0, "/content/seq2seq-fingerprint_MI")

from unsupervised.seq2seq_model import FingerprintFetcher

In [32]:
# Define our seq2seq featurizer.

from rdkit import Chem

class Seq2seqFeaturizer(Featurizer):
    """Seq2seq Featurizer."""

    def __init__(self, model_dir, vocab_dir):
        """Define the seq2seq feature."""
        self.fetcher = FingerprintFetcher(model_dir, vocab_dir)

    def _featurize(self, mol):
        """
        Calculate features for a single molecule.
        Parameters
        ----------
        mol : RDKit Mol
            Molecule.
        """
        # This is a bit hacky. I have no idea why we have to start from mol instead of original smile.
        smile = Chem.MolToSmiles(mol)
        fp, _ = self.fetcher.decode(smile)
        return fp

In [35]:
!git status

fatal: not a git repository (or any of the parent directories): .git


In [33]:
# Initailize the featurizer and cache it.
sess = tf.compat.v1.InteractiveSession()
#model directory and vocab directory
featurizer = Seq2seqFeaturizer("/content/initialisation/test/gru-4-256", "/content/initialisation/pretrain/pm2.vocab")

AttributeError: module 'tensorflow' has no attribute 'contrib'

In [22]:
# Build up specific model builder.

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR, NuSVR
# Building scikit random forest model

# Use this class to select different models for different task/dataset/split.
class SKLearnModelSelector(object):

    DATASET_MAPPING = {
        "delaney": {
            "index": (RandomForestRegressor, {}),
            "random": (RandomForestRegressor, {}),
            "scaffold": (RandomForestRegressor, {})
        },
        "sampl": {
            "index": (RandomForestRegressor, {}),
            "random": (RandomForestRegressor, {}),
            "scaffold": (RandomForestRegressor, {})
        }
    }

    def __init__(self, dataset, split):
        """Input dataset and split."""
        self.dataset = dataset
        self.split = split

    def __call__(self, task):
        model_class, model_hparam = self.DATASET_MAPPING[self.dataset][self.split]
        sklearn_model = model_class(**model_hparam)
        return dc.models.sklearn_models.SklearnModel(sklearn_model, task)


In [ ]:
from deepchem.molnet.run_benchmark import load_dataset, benchmark_model
from itertools import product

metric = [dc.metrics.Metric(dc.metrics.pearson_r2_score, np.mean)]

datasets = [
    # 'kaggle', # does not allow featurizer input
    'delaney', # we discard it here to save the testing time.
    # 'nci', # Could be very large. discard it if we do not use it.
    # 'chembl', # too long to run, discard.
    'sampl'
]
splits = [
    "index",
    "random",
    "scaffold"
]

for dataset, split in product(datasets, splits):
    print("="*80)
    print("Dataset: %s, split: %s" % (dataset, split))
    tasks, all_datasets, transformers = load_dataset(dataset, featurizer, split)
    reg_model = dc.models.multitask.SingletaskToMultitask(tasks, SKLearnModelSelector(dataset, split))
    train, val, test, t = benchmark_model(reg_model, all_datasets, transformers, metric, test=True)
    print(train, val, test)
    print("t = %.10f" % t)
    print("="*80)

Dataset: delaney, split: index
-------------------------------------
Loading dataset: delaney
-------------------------------------
Splitting function: index
About to featurize Delaney dataset.
Loading raw samples now.
shard_size: 8192
About to start loading CSV from /tmp/delaney-processed.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
TIMING: featurizing shard 0 took 50.006 s
TIMING: dataset construction took 50.163 s
Loading dataset from disk.
About to transform data
TIMING: dataset construction took 0.174 s
Loading dataset from disk.
TIMING: dataset construction took 0.144 s
Loading dataset from disk.
TIMING: dataset construction took 0.038 s
Loading dataset from disk.
TIMING: dataset construction took 0.040 s
Loading dataset from disk.
About to initialize singletask to multitask model
Initializing directory for task measured log solubility in mols per litre
About to create task-specific datasets
Splitting multitask dataset into singletask datasets
TI

In [ ]:
sess.close()